In [45]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
from scipy.sparse import coo_matrix

In [46]:
class FastGCNLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, activation=None, dropout_rate=0.0, **kwargs):
        super(FastGCNLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = activation
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
        )
        super(FastGCNLayer, self).build(input_shape)

    def call(self, features, adj, sampled_nodes, training=False):
        # Convert sampled_nodes to int64 to match adj.indices data type
        sampled_nodes = tf.cast(sampled_nodes, dtype=tf.int64)

        # Filter adjacency matrix rows and columns for sampled nodes
        row_mask = tf.reduce_any(adj.indices[:, 0:1] == sampled_nodes, axis=1)
        col_mask = tf.reduce_any(adj.indices[:, 1:2] == sampled_nodes, axis=1)
        sampled_adj_indices = tf.boolean_mask(adj.indices, tf.logical_and(row_mask, col_mask))
        sampled_adj_values = tf.boolean_mask(adj.values, tf.logical_and(row_mask, col_mask))

        # Remap node indices to the range 0 to len(sampled_nodes) - 1
        unique_nodes, remapped_indices = tf.unique(tf.reshape(sampled_adj_indices, [-1]))

        # Explicitly cast tf.range to int64 to match expected data type
        node_map = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(unique_nodes, tf.range(tf.size(unique_nodes), dtype=tf.int64)), # Cast to int64
            -1
        )
        remapped_indices = tf.reshape(node_map.lookup(tf.reshape(sampled_adj_indices, [-1])), tf.shape(sampled_adj_indices))

        # Get the actual number of valid sampled nodes
        num_valid_nodes = tf.shape(unique_nodes)[0]

        sampled_adj = tf.sparse.SparseTensor(
            indices=remapped_indices, #Use remapped indices here
            values=sampled_adj_values,
            # Use num_valid_nodes instead of len(sampled_nodes)
            dense_shape=[num_valid_nodes, num_valid_nodes]
        )
        sampled_adj = tf.sparse.reorder(sampled_adj)

        # Extract features for sampled nodes
        # Remap sampled_nodes to the new range (0 to num_samples - 1)
        # This is crucial for the second layer
        sampled_nodes_remapped = node_map.lookup(sampled_nodes)

        # Filter out invalid indices (-1)
        valid_indices_mask = tf.where(sampled_nodes_remapped >= 0)
        sampled_nodes_remapped = tf.gather_nd(sampled_nodes_remapped, valid_indices_mask)

        # Gather features for valid indices only
        sampled_features = tf.gather(features, sampled_nodes_remapped, validate_indices=False)

        # Perform sparse neighborhood aggregation
        aggregated_features = tf.sparse.sparse_dense_matmul(sampled_adj, sampled_features)
        aggregated_features = tf.matmul(aggregated_features, self.kernel)

        # Apply activation and dropout
        if self.activation:
            aggregated_features = self.activation(aggregated_features)
        return self.dropout(aggregated_features, training=training)

# FastGCN Model
class FastGCN(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super(FastGCN, self).__init__()
        self.layer1 = FastGCNLayer(hidden_dim, activation=tf.nn.relu, dropout_rate=dropout_rate)
        self.layer2 = FastGCNLayer(output_dim, activation=None, dropout_rate=dropout_rate)

    def call(self, features, adj, sampled_nodes, training=False):
        x = self.layer1(features, adj, sampled_nodes, training=training)
        x = self.layer2(x, adj, sampled_nodes, training=training)
        return x


In [47]:
# Importance Sampling Function
def importance_sampling(adj, num_samples):
    degrees = tf.sparse.reduce_sum(adj, axis=1) + 1e-5  # Avoid division by zero
    probabilities = degrees / tf.reduce_sum(degrees)
    sampled_nodes = np.random.choice(adj.shape[0], num_samples, replace=False, p=probabilities.numpy())
    return tf.constant(sampled_nodes, dtype=tf.int32)


# Training Function
def train(model, features, adj, labels, train_mask, num_epochs=50, num_samples=500, learning_rate=0.01):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            # Importance sampling
            sampled_nodes = importance_sampling(adj, num_samples)

            # Forward pass
            logits = model(features, adj, sampled_nodes, training=True)

            # Get the valid sampled nodes from the first layer
            # Ensure valid_sampled_nodes are less than the size of logits
            valid_sampled_nodes = tf.where(sampled_nodes < tf.shape(logits)[0])[:, 0]
            valid_sampled_nodes = tf.cast(valid_sampled_nodes, dtype=tf.int32)

            # Further filter to ensure they are within the range of logits
            valid_indices = tf.where(valid_sampled_nodes < tf.shape(logits)[0])
            valid_sampled_nodes = tf.gather_nd(valid_sampled_nodes, valid_indices)

            # Compute loss for valid sampled nodes
            sampled_labels = tf.gather(labels, valid_sampled_nodes)
            loss = loss_fn(sampled_labels, tf.gather(logits, valid_sampled_nodes))

        # Backward pass
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy():.4f}")

In [48]:
def load_cora():
    num_nodes = 2708
    num_features = 1433
    num_classes = 7

    # Simulate feature matrix and labels
    features = np.random.rand(num_nodes, num_features).astype(np.float32)
    labels = np.random.randint(0, num_classes, size=(num_nodes,), dtype=np.int32)

    # Simulate adjacency matrix (sparse graph structure)
    row = np.random.randint(0, num_nodes, size=10000)
    col = np.random.randint(0, num_nodes, size=10000)
    data = np.ones(len(row), dtype=np.float32)
    adjacency = coo_matrix((data, (row, col)), shape=(num_nodes, num_nodes))

    # Define train mask (e.g., first 140 nodes for training)
    train_mask = np.zeros(num_nodes, dtype=bool)
    train_mask[:140] = True

    return features, labels, adjacency, train_mask

# Load data
features, labels, adjacency, train_mask = load_cora()

# Convert adjacency matrix to TensorFlow SparseTensor
adj_tensor = tf.sparse.SparseTensor(
    indices=np.vstack((adjacency.row, adjacency.col)).T,
    values=adjacency.data,
    dense_shape=adjacency.shape
)
adj_tensor = tf.sparse.reorder(adj_tensor)

# Model parameters
input_dim = features.shape[1]
hidden_dim = 16
output_dim = labels.max() + 1

# Initialize and train the model
model = FastGCN(input_dim, hidden_dim, output_dim, dropout_rate=0.5)
train(model, features, adj_tensor, labels, train_mask, num_epochs=50, num_samples=500, learning_rate=0.01)


Epoch 1/50, Loss: 2.1464
Epoch 2/50, Loss: 7.9064
Epoch 3/50, Loss: 5.3341
Epoch 4/50, Loss: 1.9459
Epoch 5/50, Loss: 1.9459
Epoch 6/50, Loss: 1.9459
Epoch 7/50, Loss: 1.9459
Epoch 8/50, Loss: 1.9459
Epoch 9/50, Loss: 1.9459
Epoch 10/50, Loss: 1.9459
Epoch 11/50, Loss: 1.9459
Epoch 12/50, Loss: 1.9459
Epoch 13/50, Loss: 1.9459
Epoch 14/50, Loss: 1.9459
Epoch 15/50, Loss: 1.9459
Epoch 16/50, Loss: 1.9459
Epoch 17/50, Loss: 1.9459
Epoch 18/50, Loss: 1.9459
Epoch 19/50, Loss: 1.9459
Epoch 20/50, Loss: 1.9459
Epoch 21/50, Loss: 1.9459
Epoch 22/50, Loss: 1.9459
Epoch 23/50, Loss: 1.9459
Epoch 24/50, Loss: 1.9459
Epoch 25/50, Loss: 1.9459
Epoch 26/50, Loss: 1.9459
Epoch 27/50, Loss: 1.9459
Epoch 28/50, Loss: 1.9459
Epoch 29/50, Loss: 1.9459
Epoch 30/50, Loss: 1.9459
Epoch 31/50, Loss: 1.9459
Epoch 32/50, Loss: 1.9459
Epoch 33/50, Loss: 1.9459
Epoch 34/50, Loss: 1.9459
Epoch 35/50, Loss: 1.9459
Epoch 36/50, Loss: 1.9459
Epoch 37/50, Loss: 1.9459
Epoch 38/50, Loss: 1.9459
Epoch 39/50, Loss: 1.